## Load Data and Packages

In [6]:
# Import libraries and data 229623
import scipy.io as sio
import numpy as np
from scipy.signal import find_peaks
from os.path import dirname, join as pjoin
import datetime
import csv
import sys
sys.path.insert(1,'../')
import Tools.data_processing as dp
import Tools.data_movement as dm 
from auxiliary_functions import extract_runoff, get_stage_events

fDOM_raw_data = dm.read_in_preprocessed_timeseries('/Users/zachfogg/Desktop/DB-SRRW/Data/converted_data/julian_format/fDOM_raw_10.1.2011-9.4.2020.csv')
stage_data = dm.read_in_preprocessed_timeseries('/Users/zachfogg/Desktop/DB-SRRW/Data/converted_data/julian_format/stage_10.1.11-1.1.19.csv')
turb_data = dm.read_in_preprocessed_timeseries('/Users/zachfogg/Desktop/DB-SRRW/Data/converted_data/julian_format/turbidity_raw_10.1.2011_9.4.2020.csv')

dm.write_data_to_trainset(
                        fDOM_raw_data,
                        stage_data, 
                        turb_data,
                        '/Users/zachfogg/Desktop/DB-SRRW/Data/manual_annotating_data/non_annotated_data/blank_0k-100k.csv',
                        False, True,
                        0,100000)


In [10]:
from scipy.signal import find_peaks

prominence_range = [.01,None]
width_range = [None,None]
wlen = 500
distance = 1
rel_height =1.0

# data = fDOM_raw_data[:100000][:,1]

# Get list of all peaks that could possibly be sharp peaks
start = datetime.datetime.now()
stage_peaks, props = find_peaks(stage_data[:,1],
                          height = (None, None),
                          threshold = (None,None),
                          distance = distance,
                          prominence = prominence_range,
                          width = width_range,
                          wlen = wlen,
                          rel_height = rel_height)
print(datetime.datetime.now() - start)
def detected_peak_to_event(data: np.ndarray, left_ips: float, right_ips: float):
    # Use left_ips, right_ips to determine start and end of event 
    left_ips = round(left_ips)
    right_ips = round(right_ips)
    event = []
    for i in range(left_ips, right_ips+1):
        event.append(data[i])
    return np.array(event)

def detected_peak_to_3point(data: np.ndarray, peak : int):
    return np.array((data[peak-1], data[peak], data[peak+1]))
# Iterate through peaks and write to list 
stage_events = []
# for i in range(len(props['left_ips'])):
#     stage_events.append(detected_peak_to_event(stage_data, props['left_ips'][i],props['right_ips'][i]))
for peak in stage_peaks:
            stage_events.append(detected_peak_to_3point(stage_data, peak))

0:00:00.004888


In [ ]:
missing_file_path = '/Users/zachfogg/Desktop/DB-SRRW/Data/Misc/delete_date_ranges.txt'
fDOM_events = dp.delete_missing_data_events(fDOM_events,missing_file_path)
turb_events= dp.delete_missing_data_events(turb_events,missing_file_path)
stage_events = dp.delete_missing_data_events(stage_events, missing_file_path)

In [105]:
def sort_events(events):
    sorted_events = []
    prev_end = -1
    for event in events:
        i = 0
        start = event[i][0]
        end = event[-1][0]
        while start <= prev_end:
            i+=1
            if i == len(event): break
            start = event[i][0]
        if len(event[i:-1,:]) > 0:
            sorted_events.append(event[i:-1,:])
            prev_end = end
    return sorted_events

prev_start = -1
prev_end = -1
count = 0
for event in stage_events: 
    start = event[0][0]
    end = event[-1][0]
    if start < prev_end:
        count +=1
#         print(prev_end, start)
    prev_end = end

print(count, len(stage_events))

# prev_start = -1
# prev_end = -1
# count = 0

# sorted_events = sort_events(stage_events)

# for event in sorted_events: 
#     start = event[0][0]
#     end = event[-1][0]
#     if start <= prev_end:
#         count +=1
#     prev_end = end

# print(count, len(sorted_events))

0 888
